In [ ]:
import numpy as np
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
adata = sc.read('../data/all_raw_counts_clean.h5ad')

In [ ]:
pd.set_option('display.max_columns', None)  

In [ ]:
adata = adata[~adata.obs.dataset.isin(['villani_pbmc','vento_tormo_pbmc'])]

In [ ]:
adata.obs['sample_ID_lataq'] = adata.obs['batch_czi'].astype('str') + '_' + adata.obs['dataset'].astype('str') + '_' + adata.obs['subject_ID_czi'].astype('str') + '_' + adata.obs['sample_ID_czi'].astype('str') + '_' + adata.obs['assay'].astype('str') 

In [ ]:
mdata = adata.obs[['dataset','sample_ID_czi','subject_ID_czi','batch_czi','assay','sex','disease']]
mdata.drop_duplicates(inplace = True)

In [ ]:
len(set(adata.obs['sample_ID_lataq'])) # matches above computations 

In [ ]:
adata.obs['sample_ID_lataq'] = adata.obs['sample_ID_lataq'].astype('category')

In [ ]:
dataset_sample = adata.obs[['dataset','sample_ID_lataq']].drop_duplicates(subset = ['sample_ID_lataq'])

In [ ]:
dataset_sample.dataset.value_counts()

In [ ]:
len(set(adata.obs.dataset))

In [ ]:
print(len(set(mdata.sample_ID_czi)))
print(len(set(mdata.subject_ID_czi)))
print('Number of unique ids')
mdata.shape[0]

In [ ]:
# make the reference-query split

query_dataset = ['10_1126_sciimmunol_abd1554','10_1038_s41591_021_01329_2']

query_adata = adata[adata.obs.dataset.isin(query_dataset)].copy()
ref_adata = adata[~adata.obs.dataset.isin(query_dataset)].copy()

In [ ]:
len(set(query_adata.obs.sample_ID_lataq))

In [ ]:
len(set(ref_adata.obs.sample_ID_lataq))

In [ ]:
sc.pp.highly_variable_genes(
    ref_adata,
    flavor="seurat_v3",
    n_top_genes=10000,
    batch_key="batch_czi",
    subset=True
)

In [ ]:
query_adata = query_adata[:,ref_adata.var_names]

In [ ]:
query_adata

In [ ]:
ref_adata

In [ ]:
ref_adata.write('data/czi_8M_PBMC_lataq_ref_10k_loSuHanifa.h5ad')

In [ ]:
query_adata.write('data/czi_8M_PBMC_lataq_querySu_10k_loSuHanifa.h5ad')